In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import math

from sklearn.model_selection import train_test_split, RandomizedSearchCV, KFold
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR 
from sklearn.preprocessing import LabelEncoder
from sklearn.multioutput import MultiOutputRegressor

import sys
import os
from tqdm import tqdm

RAND = 10
N_FOLDS = 3
N_ITER = 300

backend_path = os.path.abspath('../backend')
sys.path.append(backend_path)
from get_metrics import get_metrics_regression, get_metrics_classification
from check_overfitting import check_overfitting

In [ ]:
# Чтение DataFrame df в файл data/df.csv
df = pd.read_csv('../data/df.csv')

In [ ]:
df.info()

In [ ]:
# создаем LabelEncoder для кодирования категориальных значений
#le = LabelEncoder()
# кодирование колонки 'country'
#le.fit(df['country'])  # обучаем энкодер на df
#df['country'] = le.fit_transform(df['country'])

# разделение данных train_test_split

In [ ]:
# признаки
X = df[['milliseconds', 'significance', 'country', 'depth', 'year', 'month', 'day', 'hour', 'minute', 'second']]

# целевые переменные
y = df[['magnitude', 'longitude', 'latitude']]

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=RAND)


In [ ]:
# создаем LabelEncoder для кодирования категориальных значений
le = LabelEncoder()

# обучаем le на тренировочных и тестовых данных
le.fit(np.concatenate((X_train['country'], X_test['country'])))

# кодирование колонки 'country' для обучения моделей
X_train['country'] = le.transform(X_train['country'])
X_test['country'] = le.transform(X_test['country'])

# Baseline 

## RandomForestRegressor

In [ ]:
# модель RandomForestRegressor
rfr = RandomForestRegressor(n_jobs=-1, random_state=RAND)
# обучаем модель
rfr.fit(X_train, y_train)
# предсказания на test 
y_pred_rfr = rfr.predict(X_test)

In [ ]:
# проверка на переобучение модели RandomForestRegressor
check_overfitting(rfr, X_train, y_train, X_test, y_test, mean_squared_error)

In [ ]:
# просмотр метрик модели RandomForestRegressor обученной на train
metrics = get_metrics_regression(y_test,
                                 y_pred = y_pred_rfr,
                                 X_test = X_test,
                                 name='RandomForestRegressor_Baseline')
metrics

## DecisionTreeRegressor

In [ ]:
# модель DecisionTreeRegressor
dtr = DecisionTreeRegressor(random_state=RAND)
# обучаем модель
dtr.fit(X_train, y_train)
# предсказания на test 
y_pred_dtr = dtr.predict(X_test)

In [ ]:
# проверка на переобучение модели DecisionTreeRegressor
check_overfitting(dtr, X_train, y_train, X_test, y_test, mean_squared_error)

In [ ]:
# просмотр метрик модели DecisionTreeRegressor обученной на train
metrics = pd.concat([
    metrics,
    get_metrics_regression(y_test,
                           y_pred = y_pred_dtr,
                           X_test = X_test,
                           name='DecisionTreeRegressor_Baseline')])
metrics

## GradientBoostingRegressor

In [ ]:
# модель GradientBoostingRegressor
gbr = MultiOutputRegressor(GradientBoostingRegressor(random_state=RAND)) 
# обучаем модель
gbr.fit(X_train, y_train)
# предсказания на test 
y_pred_gbr = gbr.predict(X_test)


In [ ]:
# проверка на переобучение модели GradientBoostingRegressor
check_overfitting(gbr, X_train, y_train, X_test, y_test, mean_squared_error)

In [ ]:
# просмотр метрик модели GradientBoostingRegressor обученной на train
metrics = pd.concat([
    metrics,
    get_metrics_regression(y_test,
                           y_pred = y_pred_gbr,
                           X_test = X_test,
                           name='GradientBoostingRegressor_Baseline')])
metrics

# RandomizedSearchCV  

## RandomForestRegressor & KFold

In [ ]:
# сетка параметров
param_grid = {
    'n_estimators': [100, 500, 1000],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 3],
    'min_weight_fraction_leaf': [0.0, 0.1, 0.5],
    'max_features': [1 , 5 , 10],
    'bootstrap': [True, False],
}

# расчет количества моделей
num_models = math.prod(len(values) for values in param_grid.values())

print(f"Количество моделей для обучения: {num_models}")

In [ ]:
# модель RandomForestRegressor
rfr = RandomForestRegressor(random_state=RAND, n_jobs=-1)

# Создание KFold
kf = KFold(n_splits=N_FOLDS, shuffle=True, random_state=RAND)

# модель RandomizedSearchCV
rsCV = RandomizedSearchCV(rfr, param_grid, cv=kf, n_iter=N_ITER, n_jobs=-1, error_score='raise',  random_state=RAND)

# производим обучение по сетке
rsCV.fit(X_train, y_train)

In [ ]:
# просмотр параметров
best_params_rfr = rsCV.best_params_
print(best_params_rfr)
print(rsCV.best_score_)

In [ ]:
# модель RandomForestRegressor с лучшими параметрами
rfr_best_params = RandomForestRegressor(**best_params_rfr, random_state=RAND)
# обучаем модель
rfr_best_params.fit(X_train, y_train)
# предсказания на test 
y_pred_rfr_best_params = rfr_best_params.predict(X_test)

In [ ]:
# проверка на переобучение модели RandomForestRegressor с лучшими параметрами
check_overfitting(rfr_best_params, X_train, y_train, X_test, y_test, mean_squared_error)

In [ ]:
# просмотр метрик модели RandomForestRegressor обученной на train с лучшими параметрами
metrics = pd.concat([
    metrics,
    get_metrics_regression(y_test,
                           y_pred = y_pred_rfr_best_params,
                           X_test = X_test,
                           name='RandomForestRegressor_best_params')])
metrics

## DecisionTreeRegressor

In [ ]:
# сетка параметров
param_grid = {
    'splitter': ['best', 'random'],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 3],
    'min_weight_fraction_leaf': [0.0, 0.1, 0.5],
    'max_features': [1 , 5 , 10],
    'max_leaf_nodes':[None, 0, 5]
}

# расчет количества моделей
num_models = math.prod(len(values) for values in param_grid.values())

print(f"Количество моделей для обучения: {num_models}")

In [ ]:
# модель DecisionTreeRegressor
dtr = DecisionTreeRegressor(random_state=RAND)

# Создание KFold
kf = KFold(n_splits=N_FOLDS, shuffle=True, random_state=RAND)

# модель RandomizedSearchCV
rsCV = RandomizedSearchCV(dtr, param_grid, cv=kf, n_iter=N_ITER, n_jobs=-1, error_score='raise',  random_state=RAND)

# производим обучение по сетке
rsCV.fit(X_train, y_train)

In [ ]:
# просмотр параметров
best_params_dtr = rsCV.best_params_
print(best_params_dtr)
print(rsCV.best_score_)

In [ ]:
# модель DecisionTreeRegressor лучшими параметрами
dtr_best_params = DecisionTreeRegressor(**best_params_dtr, random_state=RAND)
# обучаем модель
dtr_best_params.fit(X_train, y_train)
# предсказания на test 
y_pred_dtr_best_params = dtr_best_params.predict(X_test)

In [ ]:
# проверка на переобучение модели DecisionTreeRegressor с лучшими параметрами
check_overfitting(dtr_best_params, X_train, y_train, X_test, y_test, mean_squared_error)

In [ ]:
# просмотр метрик модели DecisionTreeRegressor обученной на train с лучшими параметрами
metrics = pd.concat([
    metrics,
    get_metrics_regression(y_test,
                           y_pred = y_pred_dtr_best_params,
                           X_test = X_test,
                           name='DecisionTreeRegressor_best_params')])
metrics

## GradientBoostingRegressor

In [ ]:
# сетка параметров
param_grid = {
    'estimator__learning_rate': [0,1, 0,5, 1],
    'estimator__n_estimators': [100, 500, 1000],
    'estimator__subsample': [0.0, 0.5, 1.0],
    'estimator__min_samples_split': [2, 5, 10],
    'estimator__min_samples_leaf': [1, 2, 3],
    'estimator__min_weight_fraction_leaf': [0.0, 0.1, 0.5],
    'estimator__max_depth': [None, 5, 10],
}

# расчет количества моделей
num_models = math.prod(len(values) for values in param_grid.values())

print(f"Количество моделей для обучения: {num_models}")

In [ ]:
# модель GradientBoostingRegressor
gbr = MultiOutputRegressor(GradientBoostingRegressor(random_state=RAND))

# Создание KFold
kf = KFold(n_splits=N_FOLDS, shuffle=True, random_state=RAND)

# модель RandomizedSearchCV
rsCV = RandomizedSearchCV(gbr, param_grid, cv=kf, n_iter=N_ITER, n_jobs=-1, error_score='raise', random_state=RAND)

# производим обучение по сетке
rsCV.fit(X_train, y_train)

In [ ]:
# просмотр параметров
best_params_gbr = rsCV.best_params_
print(best_params_gbr)
print(rsCV.best_score_)

In [ ]:
# модель GradientBoostingRegressor лучшими параметрами
gbr_best_params = MultiOutputRegressor(GradientBoostingRegressor(**{k.replace('estimator__', ''): v for k, v in best_params_gbr.items()}, random_state=RAND))
# обучаем модель
gbr_best_params.fit(X_train, y_train)
# предсказания на test 
y_pred_gbr_best_params = gbr_best_params.predict(X_test)

In [ ]:
# проверка на переобучение модели GradientBoostingRegressor с лучшими параметрами
check_overfitting(gbr_best_params, X_train, y_train, X_test, y_test, mean_squared_error)

In [ ]:
# просмотр метрик модели GradientBoostingRegressor обученной на train с лучшими параметрами
metrics = pd.concat([
    metrics,
    get_metrics_regression(y_test,
                           y_pred = y_pred_gbr_best_params,
                           X_test = X_test,
                           name='GradientBoostingRegressor_best_params')])
metrics

In [ ]:
#X = scaler.inverse_transform(X_scaled)

In [ ]:
# декодирование колонки 'country'
#X_train['country'] = le.inverse_transform(X_train['country'])
#X_test['country'] = le.inverse_transform(X_test['country'])